In [ ]:
#!pip install python-dotenv
#!pip install openai
#!pip install langchain
#!pip install supabase
#!pip install tiktoken

In [ ]:
import os
import openai
import datetime
from supabase.client import Client, create_client
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import SupabaseVectorStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader


In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [ ]:
openai.api_key  = os.environ['OPENAI_API_KEY']
supabase_url  = os.environ['SUPABASE_URL']
supabase_key  = os.environ['SUPABASE_SERVICE_KEY']

In [ ]:
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

In [ ]:
supabase: Client = create_client(supabase_url, supabase_key)

In [ ]:
chat = ChatOpenAI(temperature=0, openai_api_key=openai.api_key )

In [ ]:
human_message = input("What do you want a recipe for? ")

In [ ]:
messages = [
  SystemMessage(
    content="You are the head chef at an Asian cuisine restaurant, specializing in Pakistani food. You excel at making delicious dishes like biryani, karahi, and kebabs. Ask me for a recipe or any cooking-related questions!"
  ),
  HumanMessage(
    content=human_message
  ),
]
recipe_answer = chat(messages)

print(recipe_answer)

In [ ]:
file_name = "recipe_answer.txt"
with open(file_name, "w") as file:
    file.write(str(recipe_answer))  


In [ ]:
loader = TextLoader("./recipe_answer.txt")
documents = loader.load()

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


In [ ]:
embeddings = OpenAIEmbeddings()


In [ ]:
vector_store = SupabaseVectorStore.from_documents(docs, embeddings, client=supabase, table_name="documents", query_name="match_documents")

In [ ]:
query = "can you tell me the recipe of biryani?"
matched_docs = vector_store.similarity_search(query)

In [ ]:
print(matched_docs[0].page_content)